In [26]:
# Importing libraries
from pprint import pprint
import pandas as pd  
import numpy as np  
import itertools
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
%matplotlib inline
import pandas_datareader as pdr
import datetime 
import yfinance as yf
import statsmodels.api as sm
import statistics
import seaborn as sns
from scipy import stats
from scipy.stats import laplace
from scipy.stats import norm
from scipy.stats import kurtosis
import random
from scipy.stats import skew 
import pylab as pl
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn import mixture as mix
from sklearn import metrics
import operator
from nsepy import get_history
from datetime import date
from sklearn.model_selection import KFold
from nsetools import Nse


In [33]:
from nsetools import Nse
nse = Nse()
live_vix=nse.get_index_quote("INDIA VIX")
live_vix_value=list(live_vix.values())
vix=live_vix_value[1]

live_nifty=nse.get_index_quote("NIFTY 50")
live_nifty_value=list(live_nifty.values())
nifty_live=live_nifty_value[1]

def option_algo(s,iv,T):
    
    x=round(s,-2)
    y=x+100
    mid=(x+y)//2
    straddle=[]
    
    dist_1=abs(s-x)
    dist_2=abs(s-y)
    dist_3=abs(s-mid)

    list_data=[dist_1,dist_2,dist_3]
    data_min=min(list_data)
    
     
    """
    Strike Selection code
    
    """

    if data_min==dist_1:
        
        straddle.append(x)
    
    elif data_min==dist_2:
        
        straddle.append(y)
    
    else:
        
        straddle.append(mid)
    

    upper_strangle=int(straddle[0]+100)
    lower_strangle=int(straddle[0]-100)
    
    print(f"Live nifty is {s} and is {live_nifty_value[3]} % \n")
    
    print(f"Upper_strangle is {upper_strangle} \n")
    
    print(f"Straddle is {int(straddle[0])} \n")
    
    print(f"Lower_strangle is {lower_strangle} \n")
    
    """
    BlackScholes Equation Calcualtion
    
    """
    
    if T==2:
        T=690.0/1440.0
    elif T==1:
        T=345.0/1400.0
    else:
        T=T
        
    t=T/(365)
    
    v=iv*0.01*2
    
    
    """
    Option price calcultion for OTM CALL
    
    """
    d1_otm_call= (np.log(s/upper_strangle) + ((0.1 +(v*v*0.5))*t)) / (v*np.sqrt(t))
    d2_otm_call=d1_otm_call-(v*np.sqrt(t))
    
    call_price_otm= (s*norm.cdf(d1_otm_call)) - upper_strangle*norm.cdf(d2_otm_call)*np.exp(-0.1*t)    
    call_price_otm=round(call_price_otm,2)
    
    print(f"OTM Call price for strike {upper_strangle} is {call_price_otm} \n")
    
    """
     Option price calcultion for OTM PUT
     
    """
    d1_otm_put= (np.log(s/lower_strangle) + ((0.1 +(v*v*0.5))*t)) / (v*np.sqrt(t))
    d2_otm_put=d1_otm_put-(v*np.sqrt(t))
    
    put_price_otm= (-s*norm.cdf(-d1_otm_put)) + lower_strangle*norm.cdf(-d2_otm_put)*np.exp(-0.1*t)
    put_price_otm=round(put_price_otm,2)
    
    print(f"OTM PUT price for strike {lower_strangle} is {put_price_otm} \n")
    
    """
    Option price calcultion for ATM CALL
    
    """
    d1_atm_call= (np.log(s/straddle[0]) + ((0.1 +(v*v*0.5))*t)) / (v*np.sqrt(t))
    d2_atm_call=d1_atm_call-(v*np.sqrt(t))
    
    
    call_price_atm= (s*norm.cdf(d1_atm_call)) - straddle[0]*norm.cdf(d2_atm_call)*np.exp(-0.1*t)    
    call_price_atm=round(call_price_atm,2)
    
    print(f"ATM Call price for strike {straddle[0]} is {call_price_atm} \n")
    
    """
    Option price calcultion for ATM PUT
    
    """
    
    d1_atm_put= (np.log(s/straddle[0]) + ((0.1 +(v*v*0.5))*t)) / (v*np.sqrt(t))
    d2_atm_put=d1_atm_put-(v*np.sqrt(t))
    
    put_price_atm= (-s*norm.cdf(-d1_atm_put)) + straddle[0]*norm.cdf(-d2_atm_put)*np.exp(-0.1*t)
    put_price_atm=round(put_price_atm,2)
    
    print(f"ATM PUT price for strike {straddle[0]} is {put_price_atm} \n")
    
    """
    Lot size selection
    
    """
    total_premium=((call_price_otm + put_price_otm)*150) + ((call_price_atm + put_price_atm)*75)
    
    max_loss=(total_premium/2)
    
    lot_size=1_00_000/max_loss
    lot_size=int(round(lot_size,0))
    
    max_loss=lot_size*max_loss
    
    print(f"Max Loss is {-max_loss} and lot size to trade is {lot_size} and margin required is {lot_size*1_50_000 + 1_00_000:,}")
        
    return

option_algo(nifty_live,vix,2)

Live nifty is 11630.65 and is 1.11 % 

Upper_strangle is 11750 

Straddle is 11650 

Lower_strangle is 11550 

OTM Call price for strike 11750 is 22.95 

OTM PUT price for strike 11550 is 32.16 

ATM Call price for strike 11650.0 is 56.92 

ATM PUT price for strike 11650.0 is 74.74 

Max Loss is -99775.5 and lot size to trade is 11 and margin required is 1,750,000
